In [1]:
from dataset import tweetProccess
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import nltk, re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.sentiment.vader import SentimentIntensityAnalyzer as si


'''gets the data'''
dataHandler = tweetProccess.FileTreat()
content = dataHandler.getTweets()

no_features = 2000


#Use to download nltk content
#nltk.download()

'''defining a function to remove numbers'''
def tokenize(text):
    tokenized = []
    for word in nltk.word_tokenize(text):
        if re.search('[a-zA-Z]', word):
            tokenized.append(word.lower())
            
    return tokenized

'''defining a function to tokenize and get only vebs and adjectivies'''
def tokenizeStructures(text):
    #gets the type of each word in the text
    tags = pos_tag(word_tokenize(text))
    
    interesting = []
    for tag in tags:
        #removes tweet links
        if re.search('t.co+', tag[0]):
            continue
        
        if (tag[1] == 'VERB' or tag[1] == 'JJ') and len(tag[0]) > 2:
            interesting.append(tag[0])
    
    return interesting

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
        
def get_neg_matrix(texts):
    analyser = si()
    vec = []
    for sentense in texts:
        #vec.append([analyser.polarity_scores(sentense)['neg']])
        vec.append([analyser.polarity_scores(sentense)['neg'], analyser.polarity_scores(sentense)['pos']])
    return vec

#print(get_neg_matrix(content))

{'neu': 0.656, 'pos': 0.344, 'neg': 0.0, 'compound': 0.6369}


In [2]:
cmps = 5

lda = LatentDirichletAllocation(n_components= 5, max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
tf_vectorizer = CountVectorizer(tokenizer=tokenizeStructures, max_features=no_features, stop_words='english')

tf = tf_vectorizer.fit_transform(content)
tf_feature_names = tf_vectorizer.get_feature_names()
topics = lda.fit(tf)

In [3]:
print_top_words(lda, tf_feature_names, 10)

Topic #0: good ive sad real true old big long wish amazing
Topic #1: new bad little beautiful sure nice sick easy shit skinny
Topic #2: great hard strong important free dead perfect fucking single social
Topic #3: dont mental fat ready ill different favorite tired proud past
Topic #4: happy wrong right high black white awesome cute dear -cq


In [4]:
print(lda.components_)

[[  2.01067983e-01   2.00289652e-01   2.00739349e-01 ...,   2.00677054e-01
    2.01276625e-01   2.00188009e-01]
 [  2.01221048e-01   2.00316574e-01   2.00065631e-01 ...,   2.00033624e-01
    6.46143885e+00   1.58812453e+01]
 [  2.01283171e-01   2.06397326e-01   2.00841381e-01 ...,   2.00157589e-01
    2.00026121e-01   2.00204496e-01]
 [  2.00892176e-01   2.00336122e-01   2.00840689e-01 ...,   2.00035799e-01
    2.00521878e-01   2.00203156e-01]
 [  2.77531932e+02   2.93866558e+01   2.37068689e+01 ...,   2.66227830e+01
    2.00025684e-01   2.00211656e-01]]


In [5]:
#from sklearn.metrics.pairwise import cosine_similarity
#dist = 1 - cosine_similarity(tf)

#print(dist)